## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
%matplotlib inline

In [2]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def get_camera_calibration_params(path, x, y):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((y*x,3), np.float32)
    objp[:,:2] = np.mgrid[0:x,0:y].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(path)

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        shape = img.shape[-2::-1]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    return objpoints, imgpoints, shape

## Calibrate the camera

In [3]:
objpoints, imgpoints, shape = get_camera_calibration_params('../camera_cal/calibration*.jpg', 9, 6)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)

## Undistort images

In [4]:
# Make a list of calibration images
images = glob.glob('../camera_calout/calibration*.jpg')

if not os.path.exists('../camera_calout_undis/'):
    os.makedirs('../camera_calout_undis/')
    
for fname in images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    
    save_fname = fname.replace("camera_calout", "camera_calout_undis")
    cv2.imwrite(save_fname, dst)

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [5]:
def threshold_binary(img, s_thresh=(170, 255), sx_thresh=(30, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Combine both
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary
    

## Apply a perspective transform to rectify binary image ("birds-eye view")

In [6]:
def warper(img, src, dst):

    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image

    return warped

In [7]:
import matplotlib.image as mpimg

image = mpimg.imread('../test_images_out/straight_lines1.jpg')
imshape = image.shape

undistorted = cv2.undistort(image, mtx, dist, None, mtx)

src_pts = np.float32([[(260, 680), (568, 470), (717, 470),  (1043, 680)]])
dst_pts = np.float32([[(200, 680), (200, 0), (1000, 0), (1000, 680)]])

cv2.polylines(undistorted, np.int_(src_pts), True, [255,0,0], 10)

#plt.imshow(undistorted)
mpimg.imsave('../test_images_out/straight_lines1_draw_region.jpg', undistorted)

warped = warper(undistorted, src_pts, dst_pts)
plt.imshow(warped)
mpimg.imsave('../test_images_out/straight_lines1_warped.jpg', warped)

## Detect lanes

In [8]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/800 # meters per pixel in x dimension 

def find_lane_pixels(binary_warped, draw_rect = True):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        if draw_rect:
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped, draw_rect = True, draw_points = True):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped, draw_rect)
    #plt.imshow(out_img)
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    ## Visualization ##
    # Colors in the left and right lane regions
    if draw_points:
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]
    
    left_fit_real = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_real = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)
    
    return out_img, ploty, left_fit, right_fit, left_fit_real, right_fit_real


image = mpimg.imread('../test_images_out/straight_lines1.jpg')
imshape = image.shape

undistorted = cv2.undistort(image, mtx, dist, None, mtx)
binary = threshold_binary(undistorted)
src_pts = np.float32([[(260, 680), (568, 470), (717, 470),  (1043, 680)]])
dst_pts = np.float32([[(200, 680), (200, 0), (1000, 0), (1000, 680)]])
warped = warper(binary, src_pts, dst_pts)
out_img, ploty, left_fit, right_fit, left_fit_real, right_fit_real = fit_polynomial(warped, draw_rect = False, draw_points = False)


left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(out_img, np.int_([pts]), (0,255, 0))
plt.imshow(out_img)
mpimg.imsave('../test_images_out/straight_lines1_warped_lane_marking.jpg', out_img)

## Determine curvature

In [9]:
def measure_curvature_real(ploty, left_fit_cr, right_fit_cr):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/800 # meters per pixel in x dimension
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def measure_displacement_real(ploty, left_fit, right_fit, width):
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = 3.7/800 # meters per pixel in x dimension
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Determine vehicle position 
    leftx_maxy = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    rightx_maxy = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]
    displacement = (leftx_maxy + rightx_maxy - width) / 2
    return displacement * xm_per_pix

def draw_text_on_picture(result, left_curvature, right_curvature, displacement):
    cv2.putText(result, 'Left curvature: {:.0f} m'.format(left_curvature), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, 'Right curvature: {:.0f} m'.format(right_curvature), (50, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    disp_str = 'Vehicle is {:f} m left'.format(displacement) if displacement > 0 else 'Vehicle is {:f} m right'.format(-displacement)
    cv2.putText(result, disp_str, (50, 150), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)

left_curverad, right_curverad = measure_curvature_real(ploty, left_fit_real, right_fit_real)
displacement = measure_displacement_real(ploty, left_fit, right_fit, warped.shape[1])
print(left_curverad, right_curverad, displacement)

4603.60469857 3497.3254133 -0.185515600766


In [10]:
unwrapped = warper(out_img, dst_pts, src_pts)
mpimg.imsave('../test_images_out/straight_lines1_warped_lane_marking_unwrapped.jpg', unwrapped)

In [11]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)
result = weighted_img(unwrapped, undistorted)
draw_text_on_picture(result, left_curverad, right_curverad, displacement)
mpimg.imsave('../test_images_out/straight_lines1_warped_lane_marking_result.jpg', result)

## The pipeline for video lane marking 

In [12]:
def is_lane_valid(left_fit, right_fit):
    
    #Check if left and right fit returned a value
    if len(left_fit) ==0 or len(right_fit) == 0:
        status = False
    else:
        #Check distance b/w lines
        ploty = np.linspace(0, 20, num=10 )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        delta_lines = np.mean(right_fitx - left_fitx)
        if delta_lines >= 150 and delta_lines <=430: 
            status = True
        else:
            status = False
        
        # Calculate slope of left and right lanes at midpoint of y (i.e. 360)
        left = 2*left_fit[0]*360+left_fit[1]
        right = 2*right_fit[0]*360+right_fit[1]
        delta_slope_mid =  np.abs(left-right)
        
        #Check if lines are parallel at the middle
        if delta_slope_mid <= 0.1:
            status = True
        else:
            status = False
            
    return status

# Define a class to receive the characteristics of each line detection
class LaneMarker():
    def __init__(self):
        self.left_fit = None
        self.right_fit = None
        self.left_fit_real = None
        self.right_fit_real = None
    def find_lanes(self, img):
        imshape = img.shape
        undistorted = cv2.undistort(img, mtx, dist, None, mtx)
        binary = threshold_binary(undistorted)

        src_pts = np.float32([[(568, 470),(260, 680), (717, 470),  (1043, 680)]])
        dst_pts = np.float32([[(200, 0),(200, 680), (1000, 0), (1000, 680)]])
        warped = warper(binary, src_pts, dst_pts)
        out_img, ploty, left_fit, right_fit, left_fit_real, right_fit_real = fit_polynomial(warped, draw_rect = False, draw_points = False)
        
        if is_lane_valid(left_fit, right_fit):
            self.left_fit = left_fit
            self.right_fit = right_fit
            self.left_fit_real = left_fit_real
            self.right_fit_real = right_fit_real
        else:
            left_fit = self.left_fit
            right_fit = self.right_fit
            left_fit_real = self.left_fit_real
            right_fit_real = self.right_fit_real

        # Generate x and y values for plotting
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
            left_fit_real = left_fit=[1,1,0]
            right_fit_real = right_fit=[1,1,0]
            
            
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(out_img, np.int_([pts]), (0,255, 0))

        left_curverad, right_curverad = measure_curvature_real(ploty, left_fit_real, right_fit_real)
        displacement = measure_displacement_real(ploty, left_fit, right_fit, warped.shape[1])

        unwrapped = warper(out_img, dst_pts, src_pts)
        result = weighted_img(unwrapped, undistorted)
        draw_text_on_picture(result, left_curverad, right_curverad, displacement)
        return result


## Process the video

In [13]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
white_output = '../output_video/project_video.mp4'
clip1 = VideoFileClip("../project_video.mp4")
lane = LaneMarker()
white_clip = clip1.fl_image(lane.find_lanes)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ../output_video/project_video.mp4
[MoviePy] Writing video ../output_video/project_video.mp4


100%|█████████▉| 1260/1261 [04:05<00:00,  4.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_video/project_video.mp4 

CPU times: user 1min 46s, sys: 21.7 s, total: 2min 7s
Wall time: 4min 8s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))